In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
from sklearn.model_selection import train_test_split

#Read the data
path = '/kaggle/input/house-prices-advanced-regression-techniques/'
train_data = pd.read_csv(path + 'train.csv', index_col='Id')
test_data = pd.read_csv(path + 'test.csv', index_col='Id')

#Split the data
y = train_data.SalePrice
train_data.drop(['SalePrice'], axis=1, inplace=True)
X_train_full, X_valid_full, y_train, y_valid = train_test_split(train_data, y,
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

#Remove columns non-categorical and non-numerical
categorical_cols = [col for col in train_data.columns if
                    train_data[col].dtype == 'object' and
                    train_data[col].nunique() < 10]
numerical_cols = [col for col in train_data.columns if
                 train_data[col].dtype in ['int64','float64']]
my_cols = numerical_cols + categorical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_train.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1168.000000,956.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1162.000000,1168.000000,1168.000000,...,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000
mean,56.605308,69.614017,10589.672945,6.086473,5.572774,1970.890411,1984.692637,103.481067,439.890411,45.571918,...,473.632705,94.498288,48.044521,23.022260,3.218322,14.528253,2.118151,50.936644,6.301370,2007.819349
std,42.172322,22.946069,10704.180793,1.367472,1.116908,30.407486,20.684612,182.676225,435.106803,156.229962,...,209.442320,127.312017,68.619199,63.153093,27.916593,54.009608,36.482294,550.380636,2.725977,1.335971
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,20.000000,59.000000,7589.500000,5.000000,5.000000,1953.750000,1966.000000,0.000000,0.000000,0.000000,...,336.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000
50%,50.000000,69.000000,9512.500000,6.000000,5.000000,1972.000000,1993.000000,0.000000,379.500000,0.000000,...,477.500000,0.000000,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,167.750000,716.000000,0.000000,...,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,2260.000000,1120.000000,...,1390.000000,736.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000


In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
numerical_transformer = SimpleImputer()
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, numerical_cols),
    ('cat', categorical_transformer, categorical_cols)
])

In [4]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
def get_score(n_estimators, learning_rate):
    my_pipeline = Pipeline(steps=[
        ('preproc', preprocessor),
        ('model', XGBRegressor(n_estimators, learning_rate, random_state=0))
    ])
    score = -1 * cross_val_score(my_pipeline, train_data, y,
                                cv=3,
                                scoring='neg_mean_absolute_error')
    return score.mean()

In [5]:
my_pipeline = Pipeline(steps=[
        ('preproc', preprocessor),
        ('model', XGBRegressor(n_estimators=500, learning_rate=0.05, random_state=0))
    ])
my_pipeline.fit(train_data, y)
predicitons = my_pipeline.predict(test_data)

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/opt/conda/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[01:03:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [6]:
score = -1 * cross_val_score(my_pipeline, train_data, y,
                                cv=3,
                                scoring='neg_mean_absolute_error')
print(score.mean())

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[01:03:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[01:03:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[01:03:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
15962.150119053687


In [7]:
from sklearn.preprocessing import LabelEncoder
from category_encoders import TargetEncoder
from lightgbm import LGBMRegressor
numerical_transformer_2 = SimpleImputer(strategy='median')
categorical_transformer_high = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('catboost', TargetEncoder())
])
categorical_cols_2 = [col for col in train_data.columns if 
                      train_data[col].dtype == 'object' and 
                      train_data[col].nunique() >=10]
preprocessor_2 = ColumnTransformer(transformers=[
    ('num', numerical_transformer_2, numerical_cols),
    ('cat', categorical_transformer, categorical_cols),
    ('cat2', categorical_transformer_high, categorical_cols_2)
])
my_pipeline_2 = Pipeline(steps=[
        ('preproc', preprocessor_2),
        ('model', XGBRegressor(n_estimators=600, learning_rate=0.04, random_state=0))
    ])

In [8]:
'''
def get_score(n_estimators, learning_rate):
    my_pipeline = Pipeline(steps=[
        ('preproc', preprocessor_2),
        ('model', XGBRegressor(n_estimators=n_estimators, learning_rate=learning_rate, random_state=0))
    ])
    score = -1 * cross_val_score(my_pipeline, train_data, y,
                                cv=3,
                                scoring='neg_mean_absolute_error')
    return score.mean()
for lr in np.linspace(0.01, 0.1, num=10):
    print(get_score(600,lr))
'''

"\ndef get_score(n_estimators, learning_rate):\n    my_pipeline = Pipeline(steps=[\n        ('preproc', preprocessor_2),\n        ('model', XGBRegressor(n_estimators=n_estimators, learning_rate=learning_rate, random_state=0))\n    ])\n    score = -1 * cross_val_score(my_pipeline, train_data, y,\n                                cv=3,\n                                scoring='neg_mean_absolute_error')\n    return score.mean()\nfor lr in np.linspace(0.01, 0.1, num=10):\n    print(get_score(600,lr))\n"

In [9]:
score = -1 * cross_val_score(my_pipeline_2, train_data, y,
                             cv=3,
                             scoring='neg_mean_absolute_error')
print(score.mean())

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[01:03:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[01:03:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[01:03:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
15770.624992380554


In [10]:
my_pipeline_2.fit(train_data, y)
predictions = my_pipeline_2.predict(test_data)
output = pd.DataFrame({'Id': test_data.index,
                       'SalePrice': predictions})
output.to_csv('submission.csv', index=False)

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/opt/conda/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[01:04:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
